In [1]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score

KeyboardInterrupt: 

In [ ]:
# Load the earthquake dataset and create the binary target variable
df = pd.read_csv('cleaned_earthquake.csv')
df.head()

In [ ]:
# Separate the input features and target outputs
features = df.drop(['is_major'], axis=1)
target = df['is_major']

In [ ]:
# Split into training and testing sets
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2, random_state=42)

# convert to PyTorch tensors
train_features = torch.tensor(train_features.values).float()
train_target = torch.tensor(train_target.values).float().view(-1, 1)
test_features = torch.tensor(test_features.values).float()
test_target = torch.tensor(test_target.values).float().view(-1, 1)

In [ ]:
# define the neural network architecture
class EarthquakeNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(EarthquakeNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc1(x)
        hidden = self.sigmoid(hidden)
        output = self.fc2(hidden)
        output = self.sigmoid(output)
        return output

In [ ]:
# create the neural network object
input_size = features.shape[1]
hidden_size = 50
output_size = 1
net = EarthquakeNN(input_size, hidden_size, output_size)

In [ ]:
# define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.00001)

num_epochs = 1200

In [ ]:
# train the neural network
for epoch in range(num_epochs):
    # zero the gradients
    optimizer.zero_grad()

    # forward pass
    output = net(train_features)

    # calculate loss
    loss = criterion(output, train_target)

    # backward pass and optimization
    loss.backward()
    optimizer.step()

    # print the loss at every 100th epoch
    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, num_epochs, loss.item()))

In [ ]:
# evaluate the trained model on the test set
with torch.no_grad():
    start_time = time.time()
    test_output = net(test_features)
    end_time = time.time()
    test_loss = criterion(test_output, test_target)
    test_pred = (test_output >= 0.5).float()
    accuracy = (test_pred == test_target).float().mean()
    test_speed = end_time - start_time
    print('Test Loss: {:.4f}, Test Accuracy: {:.2f}%, Test Speed: {:.4f} seconds'.format(test_loss.item(), accuracy.item()*100, test_speed))